## Config

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

## Data

In [2]:
df_train = pd.read_parquet('M3_yearly_processed.parquet')
df_eval = pd.read_parquet('M3_yearly_simple_forecasts.parquet')

In [3]:
df_train.head()

,Series,N,NF,Category,Value,Date
0,1,20,6,MICRO,940.66,1975-01-01
1,1,20,6,MICRO,1084.86,1976-01-01
2,1,20,6,MICRO,1244.98,1977-01-01
3,1,20,6,MICRO,1445.02,1978-01-01
4,1,20,6,MICRO,1683.17,1979-01-01


In [4]:
df_eval.head()

,Series,Date,TimeGPT
0,1,1989-01-01,5121.754395
1,1,1990-01-01,4974.531738
2,1,1991-01-01,4750.660156
3,1,1992-01-01,4366.261719
4,1,1993-01-01,3914.425781


## Official Evaluation Metrics
Five accuracy measures:
1. Symmetric MAPE

2. Average Ranking

3. Median symmetric APE

4. Median RAE

5. RMSE (root mean squared error)

Forecasting metrics calculations with assist of github repository:
https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9

In [5]:
%run '/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/forecasting_metrics.py'

#### NB! When using methods from this file nothing is multiplied with 100 for percentage operations!

In [13]:
actuals = np.array([900, 120, 138, 155, 149]) 
forecasts = np.array([1100, 124, 132, 141, 149]) 

### Symmetric MAPE (Mean Absolute Percentage Error)
The symmetric MAPE fluctuates between -200% and 200%

In [16]:
smape_val = smape(actuals, forecasts)
print(f'sMAPE: {smape_val * 100:.2f}%')

sMAPE: 7.44%


### Median Symmetric APE (median symmetric absolute percentage error)

In [17]:
symmetric_ape = smdape(actuals, forecasts)
print(f'sMAPE: {symmetric_ape * 100:.2f}%')

sMAPE: 4.44%


### Median RAE (relative absolute error)

In [18]:
mdrae_val = mdrae(actuals, forecasts)
print(f'MDRAE: {mdrae_val:.2f}')

MDRAE: 0.17


### RMSE (Root Mean Square Error)

In [19]:
rmse_val = rmse(actuals, forecasts)
print(f'RMSE: {rmse_val:.2f}')

RMSE: 89.72


## Evaluation

### Agregated

In [22]:
eval_ex1 = df_eval.loc[df_eval['Series'] == 1]
train_ex1 = df_train.loc[df_train['Series'] == 1]

#### Function that returns all eval metrics specified in competition

In [41]:
def all_eval_metrics(merged_df):
    """
    Calculate sMAPE, sMdAPE, MdRAE, and RMSE for each series within a merged DataFrame.
    
    Args:
    - merged_df (pd.DataFrame): DataFrame containing both actual and forecasted values, 
                                with 'Series', 'Date', 'Value' (actual), and 'TimeGPT' (forecast) columns.
    
    Returns:
    - pd.DataFrame: DataFrame containing the evaluation metrics for each series, along with the series identifier.
    """
    
    # Assuming the benchmark model is a naive model (last known value)
    # For simplicity, this function uses the last available 'Value' as the benchmark for all points in a series,
    # which might not perfectly align with Naïve2 if seasonality adjustments are required.
    
    results = []  # Initialize an empty list to store the results
    
    for series_id in merged_df['Series'].unique():
        series_data = merged_df[merged_df['Series'] == series_id]
        
        # Ensure there are no missing values in the series data for calculation
        if series_data.dropna(subset=['Value', 'TimeGPT']).empty:
            continue
        
        actuals = series_data['Value'].values
        forecasts = series_data['TimeGPT'].values
        
        # Simplified benchmark (naive forecast) calculation: using the last known value
        benchmark = np.full_like(actuals, actuals[-1])
        
        # Calculate metrics
        smape_val = smape(actuals, forecasts)
        smdape_val = mdape(actuals, forecasts)
        mdrae_val = mdrae(actuals, forecasts, benchmark)
        rmse_val = rmse(actuals, forecasts)
        
        results.append({
            'Series': series_id,
            'symmetric_MAPE': smape_val,
            'Median_APE': smdape_val,
            'Median_RAE': mdrae_val,
            'RMSE': rmse_val
        })
    
    # Convert the list of results to a DataFrame
    return pd.DataFrame(results)

In [42]:
df_merged = pd.merge(df_train, df_eval, on=['Series', 'Date'], how='left')

In [43]:
df_merged1 = df_merged.dropna(subset=['Value', 'TimeGPT'])

In [49]:
eval_metrics_df = all_eval_metrics(df_merged1)

In [50]:
print(eval_metrics_df.mean().apply(lambda x: f'{x:.2f}'))

Series                   323.00
symmetric_MAPE             0.24
Median_dAPE                0.23
Median_RAE        3103526984.71
RMSE                    1593.61
dtype: object


### With time series steps

In [55]:
def eval_metrics_by_step(merged_df):
    """
    Calculate sMAPE, sMdAPE, MdRAE, and RMSE for each forecast step within each series.
    
    Args:
    - merged_df (pd.DataFrame): DataFrame containing both actual and forecasted values,
                                with 'Series', 'Date', 'Value' (actual), and 'TimeGPT' (forecast) columns.
    
    Returns:
    - pd.DataFrame: DataFrame containing the evaluation metrics for each forecast step within each series.
    """
    results = []  # Initialize an empty list to store the results
    
    for series_id in merged_df['Series'].unique():
        series_data = merged_df[merged_df['Series'] == series_id]
        
        # Calculate forecast horizon (assuming consecutive and complete forecast steps for each series)
        max_horizon = series_data.shape[0]
        
        # Iterate over each forecast step
        for step in range(1, max_horizon + 1):
            # For each step, select only rows up to the current forecast step
            step_data = series_data.head(step)
            
            # Check for sufficient data
            if step_data.dropna(subset=['Value', 'TimeGPT']).empty:
                continue
            
            actuals = step_data['Value'].values
            forecasts = step_data['TimeGPT'].values
            
            # Simplified benchmark calculation
            benchmark = np.full_like(actuals, actuals[-1])
            
            # Calculate metrics
            smape_val = smape(actuals, forecasts)
            smdape_val = mdape(actuals, forecasts)
            mdrae_val = mdrae(actuals, forecasts, benchmark)
            rmse_val = rmse(actuals, forecasts)
            
            results.append({
                'Series': series_id,
                'symmetric_MAPE': smape_val,
                'Median_APE': smdape_val,
                'Median_RAE': mdrae_val,
                'RMSE': rmse_val
            })
    
    # Convert the list of results to a DataFrame
    return pd.DataFrame(results)


In [56]:
step_wise = eval_metrics_by_step(df_merged1)


In [57]:
step_wise.head()

,Series,symmetric_MAPE,Median_APE,Median_RAE,RMSE
0,1,0.049135,0.047957,2.579956e+12,257.995605
1,1,0.130929,0.120115,5.920741e+12,856.962321
2,1,0.209179,0.192273,1.649461e+00,1412.834506
3,1,0.299526,0.250713,1.439518e+00,2129.439361
4,1,0.385484,0.309154,1.388347e+00,2768.711589


In [58]:
print(step_wise.mean().apply(lambda x: f'{x:.2f}'))

Series                      323.00
symmetric_MAPE                0.17
Median_APE                    0.17
Median_RAE        1598739399838.87
RMSE                       1064.17
dtype: object
